# 🤖 Simple Paper RAG + Knowledge Graph Chat

**Your personal AI research assistant** - Upload a paper, chat with it using natural language!

**What this does:**
- 📚 **RAG (Retrieval-Augmented Generation)**: Ask questions, get intelligent answers from the paper
- 🕸️ **Knowledge Graph**: Discover entities (authors, methods, concepts) and their relationships  
- 💬 **Chat Interface**: Natural language conversation with your paper
- 🧠 **Local AI**: Uses Ollama - completely private, no data sent anywhere

**Perfect for:** Understanding complex papers, research exploration, literature review

## 🛠️ Setup

In [ ]:
import sys
import os

# Add parent directory to path for imports
sys.path.append('..')

# Import our simple RAG + Knowledge Graph system
from src import UnifiedPaperChat, analyze_paper_with_chat
import json

print("✅ Setup complete! Ready to analyze papers with AI.")
print("🔧 Make sure Ollama is running with llama3.1:8b and nomic-embed-text models")

## 📄 Load Your Paper

**Quick start:** Use the example paper or change the path to your own PDF.

In [ ]:
# 📁 Set path to your paper (change this to your own PDF)
pdf_path = "../examples/d4sc03921a.pdf"  # Example paper
# pdf_path = "/path/to/your/paper.pdf"    # 👈 Change this to your paper

print(f"📚 Loading paper: {pdf_path}")
print("⏳ This takes 30-60 seconds (creating embeddings + extracting entities)...")
print()

# Load paper and build RAG + Knowledge Graph
chat_system = analyze_paper_with_chat(pdf_path)

print("\n🎉 Paper loaded successfully! Ready for intelligent chat.")

## 📊 Paper Overview

Get a quick overview of what was extracted from your paper.

In [ ]:
# Get paper overview
overview = chat_system.get_paper_overview()

print("📋 PAPER OVERVIEW")
print("=" * 50)
print(f"📰 Title: {overview['paper_info']['title']}")
print(f"📄 Content: {overview['paper_info']['chunks']} chunks created")
print(f"🕸️ Knowledge Graph: {overview['knowledge_graph']['total_nodes']} entities, {overview['knowledge_graph']['total_edges']} relationships")
print(f"💬 Status: {overview['status']}")

print("\n🔍 EXTRACTED ENTITIES:")
entities = chat_system.get_entities()
for category, items in entities.items():
    if items and isinstance(items, list) and len(items) > 0:
        print(f"  {category.title()}: {', '.join(items[:3])}{'...' if len(items) > 3 else ''}")
    elif items and isinstance(items, str):
        print(f"  {category.title()}: {items}")

## 💬 Chat with Your Paper

**Now the fun part!** Ask questions about your paper using natural language.

**Examples:**
- "What are the main findings?"
- "Who are the authors?"
- "What methods were used?"
- "Tell me about the results"
- "What concepts are most important?"

In [ ]:
# 🤔 Get suggested questions
suggestions = chat_system.suggest_questions()

print("💡 SUGGESTED QUESTIONS:")
for i, suggestion in enumerate(suggestions[:6], 1):
    print(f"  {i}. {suggestion}")

print("\n👆 Try any of these questions in the chat cells below!")

In [ ]:
# 💬 CHAT CELL 1 - Ask your first question
question = "What are the main findings of this paper?"  # 👈 Change this question

print(f"❓ You: {question}")
print("🤖 AI Assistant:")

response = chat_system.chat(question)
print(response['answer'])

print(f"\n📍 Source: {response.get('mode', 'unknown')} ({response.get('source', 'mixed')})")

In [ ]:
# 💬 CHAT CELL 2 - Ask about methods
question = "What methods were used in this research?"  # 👈 Change this question

print(f"❓ You: {question}")
print("🤖 AI Assistant:")

response = chat_system.chat(question)
print(response['answer'])

print(f"\n📍 Source: {response.get('mode', 'unknown')} ({response.get('source', 'mixed')})")

In [ ]:
# 💬 CHAT CELL 3 - Ask about entities/relationships
question = "Who are the authors and what are their contributions?"  # 👈 Change this question

print(f"❓ You: {question}")
print("🤖 AI Assistant:")

response = chat_system.chat(question)
print(response['answer'])

print(f"\n📍 Source: {response.get('mode', 'unknown')} ({response.get('source', 'mixed')})")

In [ ]:
# 💬 CHAT CELL 4 - Custom question
question = "YOUR QUESTION HERE"  # 👈 Write your own question

print(f"❓ You: {question}")
print("🤖 AI Assistant:")

response = chat_system.chat(question)
print(response.get('answer', response.get('error', 'No response')))

if 'answer' in response:
    print(f"\n📍 Source: {response.get('mode', 'unknown')} ({response.get('source', 'mixed')})")

## 🕸️ Explore Knowledge Graph

**Discover relationships** between entities in your paper.

In [ ]:
# 🔍 Explore a specific entity
entities = chat_system.get_entities()

# Pick an interesting entity to explore
entity_to_explore = None

# Try to find an interesting entity
for category, items in entities.items():
    if items and isinstance(items, list) and len(items) > 0:
        entity_to_explore = items[0]
        break

if entity_to_explore:
    print(f"🔍 Exploring entity: '{entity_to_explore}'")
    
    exploration = chat_system.explore_entity(entity_to_explore)
    
    if 'error' not in exploration:
        print(f"\n📊 Entity: {exploration['node']}")
        print(f"📁 Category: {exploration['category']}")
        print(f"🔗 Connections: {exploration['total_connections']}")
        
        if exploration['connections']:
            print("\n🕸️ RELATIONSHIPS:")
            for conn in exploration['connections']:
                print(f"  • {conn['relationship']} → {conn['target']} ({conn['category']})")
        
        if 'rag_context' in exploration:
            print(f"\n💬 Context from paper:")
            print(exploration['rag_context'][:300] + "...")
    else:
        print(f"❌ Could not explore entity: {exploration['error']}")
else:
    print("❌ No entities found to explore")

## 📈 Knowledge Graph Summary

See the overall structure of relationships in your paper.

In [ ]:
# 📊 Get knowledge graph statistics
graph_summary = chat_system.kg.get_graph_summary()

if 'error' not in graph_summary:
    print("🕸️ KNOWLEDGE GRAPH SUMMARY")
    print("=" * 40)
    print(f"📊 Total entities: {graph_summary['total_nodes']}")
    print(f"🔗 Total relationships: {graph_summary['total_edges']}")
    print(f"🎯 Graph density: {graph_summary['graph_density']:.3f}")
    
    print("\n📁 ENTITIES BY CATEGORY:")
    for category, count in graph_summary['nodes_by_category'].items():
        print(f"  {category.title()}: {count}")
    
    print("\n⭐ MOST CONNECTED ENTITIES:")
    for item in graph_summary['most_connected'][:5]:
        print(f"  • {item['node']} ({item['connections']:.2f} connection strength)")
else:
    print(f"❌ Graph summary not available: {graph_summary['error']}")

## 📝 Get Citation

**Need to cite this paper?** Get a properly formatted citation.

In [ ]:
# 📝 Get citation for the paper
paper_info = chat_system.rag.paper_data

print("📝 CITATION")
print("=" * 30)
print(f"**Title:** {paper_info['title']}")
print(f"**Citation:** {paper_info['citation']}")

print("\n📋 **Copy-paste ready:**")
print(f"{paper_info['citation']}")

## 🚀 What's Next?

**You now have a complete RAG + Knowledge Graph system for paper analysis!**

**What you can do:**
- 📚 **Load different papers** - Change the `pdf_path` and run the loading cell
- 💬 **Ask more questions** - Use the chat cells with your own questions
- 🔍 **Explore entities** - Investigate specific concepts, methods, or authors
- 🕸️ **Discover relationships** - Find connections between different parts of the paper

**Advanced usage:**
- **Batch analysis**: Load multiple papers (one at a time) and compare insights
- **Research exploration**: Use this for literature review and understanding complex topics
- **Methodology understanding**: Deep dive into experimental approaches and techniques

**Tips:**
- Be specific in your questions for better answers
- Try both factual questions ("What methods...") and analytical ones ("How do...") 
- Explore entities to understand the paper's structure
- The system gets smarter as you interact with it!

**🎉 Happy researching with AI!**